In [7]:
# built-ins
import sys
import os
sys.path.append(os.path.abspath('..'))

# modules
from container_maker_spec.service_pb2_grpc import ContainerMakerAPIStub
from container_maker_spec.types_pb2 import CreateContainerRequest
from container_maker_spec.types_pb2 import GetContainerRequest
from container_maker_spec.types_pb2 import ListContainerRequest
from container_maker_spec.types_pb2 import DeleteContainerRequest
from container_maker_spec.types_pb2 import ContainerResponse
from container_maker_spec.types_pb2 import ResourceRequirements
from src.common.utils import read_certs

# third party
import grpc

## Channels
The first step is to create GRPC channels. We have two types of channels, `insecure` and `secure`.
Based on the environment choose the right channel.

There are two cells, choose one of them.

In [8]:
# insecure channel
ic: grpc.Channel = grpc.insecure_channel("container-maker-development-service:50052")
stub: ContainerMakerAPIStub = ContainerMakerAPIStub(channel=ic)

In [9]:
# secure channel
client_key: bytes = read_certs('CLIENT_KEY', './cert/client.key')
client_cert: bytes = read_certs('CLIENT_CRT', './cert/client.crt')
ca_cert: bytes = read_certs('CA_CRT', './cert/ca.crt')

credentials: grpc.ServerCredentials = grpc.ssl_channel_credentials(
    root_certificates=ca_cert,
    private_key=client_key,
    certificate_chain=client_cert
)

sc: grpc.Channel = grpc.secure_channel("container-maker-development-service:50052", credentials)
stub: ContainerMakerAPIStub = ContainerMakerAPIStub(channel=sc)


## Operations
These are the operations:
1. Create Container
2. Save Container
3. Get Container
4. List Containers
5. Delete Container

Below we have examples for each

In [12]:
# Create Container
from container_maker_spec.types_pb2 import ExposureLevel as GRPCExposureLevel

create_container_request: CreateContainerRequest = CreateContainerRequest(
    container_name='test-ssh-container',
    network_name='browseterm-new',
    image_name='zim95/ssh_ubuntu:latest',
    exposure_level=GRPCExposureLevel.EXPOSURE_LEVEL_CLUSTER_LOCAL,
    publish_information=[
        {'publish_port': 2222, 'target_port': 22, 'protocol': 'TCP'}
    ],
    environment_variables={
        'SSH_PASSWORD': '12345678',
        'SSH_USERNAME': 'test-user',
        'CONTAINER_ID': '1234567890',
        'DB_USERNAME': 'testuser',
        'DB_PASSWORD': 'testpassword',
        'DB_NAME': 'testdb',
        'DB_HOST': 'testhost',
        'DB_PORT': '5432',
        'DB_DATABASE': 'testdatabase',
    },
    resource_requirements=ResourceRequirements(
        cpu_request='100m',
        cpu_limit='1',
        memory_request='256Mi',
        memory_limit='1Gi',
        ephemeral_request='512Mi',
        ephemeral_limit='2Gi',
        snapshot_size_limit='2Gi',
    )
)

create_container_response: ContainerResponse = stub.createContainer(create_container_request)
print(create_container_response)

container_id: "db4e4e14-2d68-414d-a4e7-def0eb1e6fbe"
container_name: "test-ssh-container-service"
container_ip: "10.106.243.38"
container_network: "browseterm-new"
ports {
  container_port: 2222
  protocol: "TCP"
}
associated_resources {
  resource_name: "test-ssh-container-pod"
  resource_type: "pod"
  associated_resources {
    resource_name: "test-ssh-container-pod"
    resource_type: "pod_container"
    container_resources {
      cpu_request: "100m"
      cpu_limit: "1"
      memory_request: "256Mi"
      memory_limit: "1Gi"
      ephemeral_request: "512Mi"
      ephemeral_limit: "2Gi"
      snapshot_size_limit: "2Gi"
    }
  }
  associated_resources {
    resource_name: "snapshot-sidecar"
    resource_type: "pod_container"
    container_resources {
      cpu_request: "100m"
      cpu_limit: "1"
      memory_request: "256Mi"
      memory_limit: "1Gi"
      ephemeral_request: "512Mi"
      ephemeral_limit: "2Gi"
      snapshot_size_limit: "2Gi"
    }
  }
  associated_resources {
  

In [13]:
# save container
from container_maker_spec.types_pb2 import SaveContainerRequest

save_container_request: SaveContainerRequest = SaveContainerRequest(
    container_id='db4e4e14-2d68-414d-a4e7-def0eb1e6fbe',
    network_name='browseterm-new',
)

save_container_response = stub.saveContainer(save_container_request)
print(save_container_response)

saved_pods {
  pod_name: "test-ssh-container-pod"
  namespace_name: "browseterm-new"
  image_name: "test-ssh-container-pod-image:latest"
}



In [14]:
# Get Container
get_container_request: GetContainerRequest = GetContainerRequest(
    container_id='db4e4e14-2d68-414d-a4e7-def0eb1e6fbe', network_name='browseterm-new'
)
get_container_response = stub.getContainer(get_container_request)
print(get_container_response)

container_id: "db4e4e14-2d68-414d-a4e7-def0eb1e6fbe"
container_name: "test-ssh-container-service"
container_ip: "10.106.243.38"
container_network: "browseterm-new"
ports {
  container_port: 2222
  protocol: "TCP"
}
associated_resources {
  resource_name: "test-ssh-container-pod"
  resource_type: "pod"
  associated_resources {
    resource_name: "test-ssh-container-pod"
    resource_type: "pod_container"
    container_resources {
      cpu_request: "100m"
      cpu_limit: "1"
      memory_request: "256Mi"
      memory_limit: "1Gi"
      ephemeral_request: "512Mi"
      ephemeral_limit: "2Gi"
      snapshot_size_limit: "2Gi"
    }
  }
  associated_resources {
    resource_name: "snapshot-sidecar"
    resource_type: "pod_container"
    container_resources {
      cpu_request: "100m"
      cpu_limit: "1"
      memory_request: "256Mi"
      memory_limit: "1Gi"
      ephemeral_request: "512Mi"
      ephemeral_limit: "2Gi"
      snapshot_size_limit: "2Gi"
    }
  }
  associated_resources {
  

In [15]:
# list containers
list_container_request: ListContainerRequest = ListContainerRequest(network_name='browseterm-new')
list_container_response = stub.listContainer(list_container_request)
print("List container response: ", list_container_response)

List container response:  containers {
  container_id: "23b7f240-b937-4cd8-8c5e-8509bbb3b7d7"
  container_name: "browseterm-pg-service"
  container_ip: "10.103.22.3"
  container_network: "browseterm-new"
  ports {
    container_port: 5432
    protocol: "TCP"
  }
  associated_resources {
    resource_name: "browseterm-pg"
    resource_type: "pod"
    associated_resources {
      resource_name: "postgres"
      resource_type: "pod_container"
      container_resources {
      }
    }
  }
}
containers {
  container_id: "1fe36230-0f40-4081-83d3-873eaa42d985"
  container_name: "browseterm-redis-service"
  container_ip: "10.107.164.21"
  container_network: "browseterm-new"
  ports {
    container_port: 6379
    protocol: "TCP"
  }
  associated_resources {
    resource_name: "browseterm-redis"
    resource_type: "pod"
    associated_resources {
      resource_name: "redis"
      resource_type: "pod_container"
      container_resources {
      }
    }
  }
}
containers {
  container_id: "52524b1

In [6]:
# delete container
delete_container_request: DeleteContainerRequest = DeleteContainerRequest(
    container_id='50bdfa86-82ab-43af-8067-2586861019c2', network_name='browseterm-new'
)
delete_container_response = stub.deleteContainer(delete_container_request)
print(delete_container_response)

container_id: "50bdfa86-82ab-43af-8067-2586861019c2"
status: "Deleted"

